<a href="https://colab.research.google.com/github/teymour-aldridge/NN/blob/master/computer_vision/boat_classifier/convnet_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial setup

## Colab-related stuff

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [16]:
LOG_DIR = '/tmp/log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2018-12-23 19:22:29--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.201.75.180, 52.203.66.95, 52.202.60.111, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.201.75.180|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  20.7MB/s    in 0.2s    

2018-12-23 19:22:29 (20.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://c24a3648.ngrok.io


In [17]:
!pip install tensorboardX

    100% |████████████████████████████████| 112kB 3.3MB/s 


In [0]:
# Install PyTorch
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import os
import shutil
import matplotlib.pyplot as plt
base_path = "/content/gdrive/My Drive/Computing/ML/Computer Vision/Boat Classification/data"

## Train-test split

In [4]:
"""
# Train/Test split
# Class names (exactly what you'd expect)
classes = ['buoy', 'cruise ship', 'ferry boat', 'freight boat', 'gondola', 'inflatable boat', 'kayak', 'paper boat', 'sailboat']
# Get the number of files in each directory
file_lengths = [len(os.listdir("/content/gdrive/My Drive/Computing/ML/Computer Vision/Boat Classification/data/" + c)) for c in classes]
# For each class
for c in classes:
  files = os.listdir(os.path.join(base_path, c)) # Get all files
  for i in range(len(files) // 5): # Get 20% of the files
    src = os.path.join(base_path, c, files[i])
    dest = os.path.abspath(os.path.join(base_path, 'test', c))
    if not os.path.exists(dest):
      os.mkdir(dest)
    shutil.move(src, dest)
"""

'\n# Train/Test split\n# Class names (exactly what you\'d expect)\nclasses = [\'buoy\', \'cruise ship\', \'ferry boat\', \'freight boat\', \'gondola\', \'inflatable boat\', \'kayak\', \'paper boat\', \'sailboat\']\n# Get the number of files in each directory\nfile_lengths = [len(os.listdir("/content/gdrive/My Drive/Computing/ML/Computer Vision/Boat Classification/data/" + c)) for c in classes]\n# For each class\nfor c in classes:\n  files = os.listdir(os.path.join(base_path, c)) # Get all files\n  for i in range(len(files) // 5): # Get 20% of the files\n    src = os.path.join(base_path, c, files[i])\n    dest = os.path.abspath(os.path.join(base_path, \'test\', c))\n    if not os.path.exists(dest):\n      os.mkdir(dest)\n    shutil.move(src, dest)\n'

In [5]:
train_path = os.path.join(base_path, 'train')
test_path = os.path.join(base_path, 'test')
classes = os.listdir(train_path)
print(classes)

['inflatable boat', 'gondola', 'sailboat', 'freight boat', 'ferry boat', 'paper boat', 'kayak', 'cruise ship', 'buoy']


## Generate indices

In [6]:
data_class = 'sailboat'
os.path.join(train_path, data_class) + '/*.jpg'

'/content/gdrive/My Drive/Computing/ML/Computer Vision/Boat Classification/data/train/sailboat/*.jpg'

In [0]:
import pandas as pd
import glob
train_index = pd.DataFrame(columns=['class', 'path'])
test_index = pd.DataFrame(columns=['class', 'path'])

for i, data_class in enumerate(os.listdir(train_path)): # For each class (there are 9)
  
  for file in glob.glob(os.path.join(train_path, data_class) + '/*.jpg'): # Get all JPEG files

    train_index = train_index.append({
        'class': i,
        'path': os.path.join(train_path, data_class, file)
    }, ignore_index = True)
    
    
for i, data_class in enumerate(os.listdir(test_path)): # For each class (there are 9)
  
  for file in glob.glob(os.path.join(test_path, data_class) + '/*.jpg'): # Get all JPEG files

    test_index = test_index.append({
        'class': i,
        'path': os.path.join(test_path, data_class, file)
    }, ignore_index=True)

## PIL error fixing

In [8]:
!pip uninstall -y Pillow
# install the new one
!pip install Pillow==5.3.0
# import the new one
import PIL
print(PIL.PILLOW_VERSION)

Uninstalling Pillow-5.3.0:
  Successfully uninstalled Pillow-5.3.0
  Using cached https://files.pythonhosted.org/packages/62/94/5430ebaa83f91cc7a9f687ff5238e26164a779cca2ef9903232268b0a318/Pillow-5.3.0-cp36-cp36m-manylinux1_x86_64.whl
5.3.0


## Define dataset

In [0]:
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

"""
  _____        _                 _   
 |  __ \      | |               | |  
 | |  | | __ _| |_ __ _ ___  ___| |_ 
 | |  | |/ _` | __/ _` / __|/ _ \ __|
 | |__| | (_| | || (_| \__ \  __/ |_ 
 |_____/ \__,_|\__\__,_|___/\___|\__|
 (Dataset class, allows for an index to be passed.)                                   
"""


class IndexedImageDataset(Dataset):
    def __init__(self, index, transform=None):
        """
        Initializes an IndexedImageDataset, which reads an index from a pandas dataframe, and acts as a database wrapper.
        :param index: A pandas dataframe storing with a list of classes and their respective files.
        :param transform: A function, to be applied to the image (any function will do).
        """
        assert isinstance(index, pd.DataFrame)  # Check if the 
        self.index = index
        self.transform = transform

    def __len__(self):
        return len(self.index.index)  # Return the length of the dataset

    def __getitem__(self, n):
        x, y = self.index['path'].iloc[n], self.index['class'].iloc[n]  # Get x and y values for the neural network
        x = io.imread(x)
        if self.transform:
            x = self.transform(x)
        return x, y


"""
  _______                   __                         
 |__   __|                 / _|                        
    | |_ __ __ _ _ __  ___| |_ ___  _ __ _ __ ___  ___ 
    | | '__/ _` | '_ \/ __|  _/ _ \| '__| '_ ` _ \/ __|
    | | | | (_| | | | \__ \ || (_) | |  | | | | | \__ \
    |_|_|  \__,_|_| |_|___/_| \___/|_|  |_| |_| |_|___/
    (Transform images)                                                  
"""


class Rescale(object):
    def __init__(self, output_size):
        """
        Rescales images down to the specified height. 
        :param output_size: The size of the image to be outputted.
        """
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, x):
        """
        Resizes an image to the desired dimensions. 
        :param x: the image to be resized
        """
        h, w = x.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        return transform.resize(x, (new_h, new_w))


In [0]:
train_image_dataset=IndexedImageDataset(train_index,
                                       transform = transforms.Compose( # Join multiple transforms together.
                                           [
                                               Rescale(output_size = (512, 512)), # Downsize all images to 512 x 512 
                                               transforms.ToTensor(),
                                               transforms.Normalize((0.5148, 0.5148, 0.5148), (0.1871, 0.1871, 0.1871))
                                           ]
                                       ))
train_image_loader = DataLoader(
    train_image_dataset,
    batch_size=4,
    num_workers=1,
    shuffle=False
)

## Normalize dataset
Note that this assumes the data is normally distributed (I haven't checked).

In [11]:
"""mean = 0.
std = 0.
nb_samples = 0.
for data, _ in train_image_loader:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples"""

'mean = 0.\nstd = 0.\nnb_samples = 0.\nfor data, _ in train_image_loader:\n    batch_samples = data.size(0)\n    data = data.view(batch_samples, data.size(1), -1)\n    mean += data.mean(2).sum(0)\n    std += data.std(2).sum(0)\n    nb_samples += batch_samples\n\nmean /= nb_samples\nstd /= nb_samples'

In [12]:
train_image_dataset[0][0].shape

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


torch.Size([3, 512, 512])

# Neural Network

In [0]:
from torch import nn
import torch.nn.functional as F

class BoatNetwork(nn.Module):
  def __init__(self, num_classes=9):
    super(BoatNetwork, self).__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=2, padding=1)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=2, padding=1)
    self.conv3 = nn.Conv2d(32, 64, kernel_size=2, padding=1)
    self.fc = nn.Linear(64*64*64, 64)
    self.fc2 = nn.Linear(64, num_classes)
  def forward(self, x):
    x = x.double()
    x = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
    x = F.max_pool2d(torch.tanh(self.conv2(x)), 2)
    x = F.max_pool2d(torch.tanh(self.conv3(x)), 2)
    x = x.reshape(x.size(0), -1)
    x = torch.tanh(self.fc(x))
    x = self.fc2(x)
    return x

In [0]:
model = BoatNetwork().double()
learning_rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 100

In [0]:
from tensorboardX import SummaryWriter
writer = SummaryWriter()

In [0]:
for epoch in range(n_epochs):
  
  for imgs, labels in train_image_loader:
    outputs = model(imgs)
    loss = loss_fn(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    writer.add_scalar('data/loss', loss, epoch)
  
  print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [0]:
import os
base_path = "/content/gdrive/My Drive/Computing/ML/Computer Vision/Boat Classification/data"
torch.save(model.state_dict(), os.path.join(base_path, 'model_weights.pt'))

In [0]:
F.log_softmax(x, dim=1)

In [0]:
x.size()